In [1]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense  
import pandas as pd

In [2]:
dataTrain=pd.read_csv("../data/newtrain.csv")
dataTest=pd.read_csv("../data/newtest.csv")
print(dataTrain.head())
print(dataTrain.shape)

   Unnamed: 0  round  venue   gf   ga   xg  xga  poss  formation    sh  ...  \
0           0     33      0  0.0  0.0  1.4  0.8  49.0          4  11.0  ...   
1           1     35      1  2.0  0.0  2.4  0.4  64.0          4  25.0  ...   
2           2     37      0  5.0  2.0  3.5  1.0  67.0          4  30.0  ...   
3           3      2      0  1.0  3.0  0.3  1.8  43.0          4   6.0  ...   
4           4      3      1  2.0  0.0  2.1  0.7  43.0          4  14.0  ...   

   ga_rolling  sh_rolling  sot_rolling  dist_rolling  fk_rolling  pk_rolling  \
0    2.333333   17.333333     6.333333     17.300000    0.333333         0.0   
1    1.333333   12.000000     3.666667     16.566667    0.333333         0.0   
2    0.000000   17.666667     6.333333     17.166667    1.000000         0.0   
3    0.666667   22.000000     8.000000     17.500000    1.333333         0.0   
4    1.666667   20.333333     8.333333     18.500000    1.666667         0.0   

   pkatt_rolling  home_code  target_away  ta

In [3]:
del dataTrain["Unnamed: 0"]
del dataTest["Unnamed: 0"]
del dataTrain["target_away"]
del dataTest["target_away"]

In [4]:
dataTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1806 entries, 0 to 1805
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   round          1806 non-null   int64  
 1   venue          1806 non-null   int64  
 2   gf             1806 non-null   float64
 3   ga             1806 non-null   float64
 4   xg             1806 non-null   float64
 5   xga            1806 non-null   float64
 6   poss           1806 non-null   float64
 7   formation      1806 non-null   int64  
 8   sh             1806 non-null   float64
 9   sot            1806 non-null   float64
 10  dist           1806 non-null   float64
 11  fk             1806 non-null   float64
 12  pk             1806 non-null   float64
 13  pkatt          1806 non-null   float64
 14  venue_code     1806 non-null   int64  
 15  opp_code       1806 non-null   int64  
 16  hour           1806 non-null   int64  
 17  day_code       1806 non-null   int64  
 18  gf_rolli

In [5]:
# Data Training
train_data = dataTrain.iloc[:, :-1].values
train_labels = dataTrain.iloc[:, -1:].values

# Data Testing
test_data = dataTest.iloc[:, :-1].values
test_labels = dataTest.iloc[:, -1:].values

In [6]:
train_data

array([[33.,  0.,  0., ...,  0.,  0.,  0.],
       [35.,  1.,  2., ...,  0.,  0.,  0.],
       [37.,  0.,  5., ...,  0.,  0.,  0.],
       ...,
       [29.,  0.,  0., ...,  0.,  0., 19.],
       [30.,  0.,  0., ...,  0.,  0., 19.],
       [31.,  1.,  1., ...,  0.,  0., 19.]])

In [7]:
# # Normalisasi Data
# scaler = MinMaxScaler()
# train_data = scaler.fit_transform(train_data)
# test_data = scaler.transform(test_data)

# Pembentukan Data Sequence
sequence_length = 1  # Panjang sequence yang akan digunakan
num_features = train_data.shape[1]

In [8]:
print(train_data[:1])

[[33.          0.          0.          0.          1.4         0.8
  49.          4.         11.          2.         17.5         0.
   0.          0.          0.          5.         13.          6.
   2.33333333  2.33333333 17.33333333  6.33333333 17.3         0.33333333
   0.          0.          0.        ]]


In [9]:
def create_sequences(data, labels, sequence_length):
    X = []
    y = []
    for i in range(len(data) - sequence_length + 1):
        X.append(data[i:i+sequence_length])
        y.append(labels[i+sequence_length-1])
    return np.array(X), np.array(y)

In [10]:
X_train, y_train = create_sequences(train_data, train_labels, sequence_length)
X_test, y_test = create_sequences(test_data, test_labels, sequence_length)

In [11]:
print(X_train[1], y_train[1])

[[35.          1.          2.          0.          2.4         0.4
  64.          4.         25.          8.         18.4         2.
   0.          0.          1.          4.         12.          6.
   1.          1.33333333 12.          3.66666667 16.56666667  0.33333333
   0.          0.          0.        ]] [1]


In [12]:
for i in range(len(X_train)):
    print(X_train[i], y_train[i])

[[33.          0.          0.          0.          1.4         0.8
  49.          4.         11.          2.         17.5         0.
   0.          0.          0.          5.         13.          6.
   2.33333333  2.33333333 17.33333333  6.33333333 17.3         0.33333333
   0.          0.          0.        ]] [0]
[[35.          1.          2.          0.          2.4         0.4
  64.          4.         25.          8.         18.4         2.
   0.          0.          1.          4.         12.          6.
   1.          1.33333333 12.          3.66666667 16.56666667  0.33333333
   0.          0.          0.        ]] [1]
[[37.          0.          5.          2.          3.5         1.
  67.          4.         30.         14.         16.6         2.
   0.          0.          0.          7.         13.          6.
   1.66666667  0.         17.66666667  6.33333333 17.16666667  1.
   0.          0.          0.        ]] [1]
[[ 2.          0.          1.          3.          0.3    

In [13]:
print(len(X_train), len(y_train))

1806 1806


In [14]:
# Pembangunan Model LSTM
model = Sequential()
model.add(LSTM(units=64, input_shape=(sequence_length, num_features)))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)

In [15]:
# Pelatihan Model
model.fit(X_train, y_train, epochs=10, batch_size=32)  # Sesuaikan dengan jumlah epoch dan ukuran batch yang optimal

Epoch 1/10
57/57 [==============================] - 3s 45ms/step - loss: 0.6041 - accuracy: 0.6811
Epoch 2/10
57/57 [==============================] - 2s 41ms/step - loss: 0.4869 - accuracy: 0.7868
Epoch 3/10
57/57 [==============================] - 2s 43ms/step - loss: 0.3867 - accuracy: 0.8494
Epoch 4/10
57/57 [==============================] - 2s 40ms/step - loss: 0.3130 - accuracy: 0.8959
Epoch 5/10
57/57 [==============================] - 2s 41ms/step - loss: 0.2516 - accuracy: 0.9219
Epoch 6/10
57/57 [==============================] - 2s 39ms/step - loss: 0.2013 - accuracy: 0.9463
Epoch 7/10
57/57 [==============================] - 2s 40ms/step - loss: 0.1687 - accuracy: 0.9557
Epoch 8/10
57/57 [==============================] - 2s 41ms/step - loss: 0.1425 - accuracy: 0.9707
Epoch 9/10
57/57 [==============================] - 2s 39ms/step - loss: 0.1234 - accuracy: 0.9701
Epoch 10/10
57/57 [==============================] - 2s 39ms/step - loss: 0.0974 - accuracy: 0.9873


In [16]:
for i in range(len(X_test)):
    print(X_test[i], y_test[i])

[[ 0.          0.          2.          0.          1.          1.2
  44.         15.         10.          2.         14.6         1.
   0.          0.          0.          6.         20.          4.
   1.66666667  2.         15.          5.         20.7         0.
   0.33333333  0.33333333  0.        ]] [1]
[[11.          1.          4.          2.          2.7         0.5
  50.         15.         19.          7.         13.          0.
   0.          0.          1.         10.         15.          5.
   2.33333333  1.         15.33333333  4.         17.56666667  0.33333333
   0.33333333  0.33333333  0.        ]] [1]
[[22.          0.          3.          0.          1.3         0.3
  57.         15.         14.          6.         14.8         0.
   0.          0.          0.          2.         17.          5.
   3.66666667  1.         18.          5.66666667 14.86666667  0.33333333
   0.33333333  0.33333333  0.        ]] [1]
[[32.          1.          2.          1.          2.6   

In [17]:
print(len(X_test), len(y_test))

609 609


In [18]:
# Testing Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")

20/20 [==============================] - 0s 20ms/step - loss: 0.1027 - accuracy: 0.9885
Loss: 0.1027
Accuracy: 0.9885


In [19]:
new = [0, 0, 2, 0, 1, 1.2, 44, 15, 10, 2, 14.6, 1, 0, 0, 0, 6, 20, 4, 1.666666667, 2, 15, 5, 20.7, 0, 0.3333333333, 0.3333333333, 0]
new = np.array(new)
new = new.reshape(1, -1)
new

array([[ 0.        ,  0.        ,  2.        ,  0.        ,  1.        ,
         1.2       , 44.        , 15.        , 10.        ,  2.        ,
        14.6       ,  1.        ,  0.        ,  0.        ,  0.        ,
         6.        , 20.        ,  4.        ,  1.66666667,  2.        ,
        15.        ,  5.        , 20.7       ,  0.        ,  0.33333333,
         0.33333333,  0.        ]])

In [20]:
new = new.reshape((1, 1, 27))
new

array([[[ 0.        ,  0.        ,  2.        ,  0.        ,
          1.        ,  1.2       , 44.        , 15.        ,
         10.        ,  2.        , 14.6       ,  1.        ,
          0.        ,  0.        ,  0.        ,  6.        ,
         20.        ,  4.        ,  1.66666667,  2.        ,
         15.        ,  5.        , 20.7       ,  0.        ,
          0.33333333,  0.33333333,  0.        ]]])

In [21]:
predictNextNumber = model.predict(new)
print(predictNextNumber)

1/1 [==============================] - 0s 45ms/step
[[0.9150219]]


In [22]:
baru = [1, 35, 1, 2, 0, 2.4, 0.4, 64, 4, 25, 8, 18.4, 2, 0, 0, 1, 4, 12, 6, 1, 1.333333333, 12, 3.666666667, 16.56666667, 0.3333333333, 0, 0]
baru = np.array(baru)
baru = baru.reshape(1, -1)
baru

array([[ 1.        , 35.        ,  1.        ,  2.        ,  0.        ,
         2.4       ,  0.4       , 64.        ,  4.        , 25.        ,
         8.        , 18.4       ,  2.        ,  0.        ,  0.        ,
         1.        ,  4.        , 12.        ,  6.        ,  1.        ,
         1.33333333, 12.        ,  3.66666667, 16.56666667,  0.33333333,
         0.        ,  0.        ]])

In [23]:
baru= baru.reshape((1, 1, 27))
baru


array([[[ 1.        , 35.        ,  1.        ,  2.        ,
          0.        ,  2.4       ,  0.4       , 64.        ,
          4.        , 25.        ,  8.        , 18.4       ,
          2.        ,  0.        ,  0.        ,  1.        ,
          4.        , 12.        ,  6.        ,  1.        ,
          1.33333333, 12.        ,  3.66666667, 16.56666667,
          0.33333333,  0.        ,  0.        ]]])

In [24]:
predictbaru = model.predict(baru)
print(predictbaru)

1/1 [==============================] - 0s 47ms/step
[[0.39935625]]


In [27]:
a = [1, 35, 1, 2, 0, 2.4, 0.4, 64, 4, 25, 8, 18.4, 2, 0, 0, 1, 4, 12, 6, 1, 1.333333333, 12, 3.666666667, 16.56666667, 0.3333333333, 0, 0]
a = np.array(a)
a= baru.reshape((1, 1, 27))
a

array([[[ 1.        , 35.        ,  1.        ,  2.        ,
          0.        ,  2.4       ,  0.4       , 64.        ,
          4.        , 25.        ,  8.        , 18.4       ,
          2.        ,  0.        ,  0.        ,  1.        ,
          4.        , 12.        ,  6.        ,  1.        ,
          1.33333333, 12.        ,  3.66666667, 16.56666667,
          0.33333333,  0.        ,  0.        ]]])

In [28]:
predictlagi = model.predict(a)
print(predictlagi)

1/1 [==============================] - 0s 47ms/step
[[0.39935625]]


In [30]:
from sklearn.metrics import classification_report

# Prediksi probabilitas pada data uji
y_pred_prob = model.predict(X_test)

# Konversi probabilitas menjadi label biner (0 atau 1) berdasarkan threshold
threshold = 0.5
y_pred_binary = np.where(y_pred_prob > threshold, 1, 0)

# Peroleh laporan klasifikasi
classification_report = classification_report(y_test, y_pred_binary)

# Tampilkan laporan klasifikasi
print(classification_report)


20/20 [==============================] - 0s 14ms/step
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       375
           1       1.00      0.97      0.98       234

    accuracy                           0.99       609
   macro avg       0.99      0.99      0.99       609
weighted avg       0.99      0.99      0.99       609

